In [ ]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns

# Load a small sample of data for initial exploration, increase amount as needed.
df = pd.read_csv('../data/raw/used_cars_data.csv', nrows=100000)

In [ ]:
# Save a smaller sample for testing purposes
sample_size = 10000

df_test_sample = df.sample(n=sample_size, random_state=42)

df_test_sample.to_csv('../data/samples/used_cars_data_sample.csv', index=False)

In [ ]:
# Loading sample data for testing
df_sample = pd.read_csv('../data/samples/used_cars_data_sample.csv')

In [ ]:
# Generate a profile of the dataset before data cleaning on high priority columns
def profile_data(df):
    profile = []

    for column in df.columns:
        profile_dict = {
            'Column Name': column,
            'Data Type': str(df[column].dtype),
            'Null Count': df[column].isna().sum(),
            'Null Percentage': round(df[column].isna().sum() / len(df) * 100, 2),
            'Unique Values': df[column].nunique(),
            'Sample Values': df[column].dropna().head(3).tolist() if not df[column].isna().all() else []
        }

        if df[column].dtype == 'object':
            sample = df[column].dropna().astype(str)
            if len(sample) > 0:
                profile_dict['has_numbers'] = sample.str.contains(r'\d').any()
                profile_dict['has_special_chars'] = sample.str.contains(r'[^a-zA-Z0-9 ]').any()
                profile_dict['potential_units'] = bool(sample.str.contains(r'\b(?:in|gal|hp|mph|seats?|ft|lbs?)\b', case=False, regex=True).any())
        
        profile.append(profile_dict)

    return pd.DataFrame(profile)

In [ ]:
# For visual used in Part D, uses matplotlib to create a visual of the top 20
# columns with greatest percentage of missing data.
profile = profile_data(df_sample)

plt.figure(figsize=(12, 6))
top_missing = profile.nlargest(20, 'Null Percentage')
plt.barh(top_missing['Column Name'], top_missing['Null Percentage'], color='blue')
plt.xlabel('Missing Data')
plt.ylabel('Column Name')
plt.title('Top 20 Columns by Missing Data Percentage')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('columns_with_missing_data.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Applying lowercase normalization to all text columns
def lowercase_text_columns(df):
    df_clean = df.copy()
    text_columns = df_clean.select_dtypes(include=['object']).columns

    # Columns to skip from lowercasing
    skip_columns = ['vin', 'listing_id', 'main_picture_url', 'dealer_zip']

    for col in text_columns:
        if col not in skip_columns:
            df_clean[col] = df_clean[col].astype(str).str.lower()
            df_clean[col] = df_clean[col].replace('nan', np.nan)

    return df_clean

In [8]:
# Apply lowercase normalization to text columns in the sample datam, later to be used on full dataset
df_cleaned = lowercase_text_columns(df_sample)

In [9]:
# Currently columns with units include text such as "in", "gal", this will be removed to leave only numeric values.
def clean_units(df):
    df_clean = df.copy()

    measurements_in = ['back_legroom', 'front_legroom', 'height', 'length', 'width', 'wheelbase', 'bed_length']
    measurements_gal = ['fuel_tank_volume']

    for column in measurements_in:
        if column in df_clean.columns:
            df_clean[column] = df_clean[column].replace(['--', '-', 'n/a'], np.nan)

            df_clean[column] = pd.to_numeric(df_clean[column].astype(str).str.replace(' in', '').str.strip(), errors='coerce')

    for column in measurements_gal:
        if column in df_clean.columns:
            df_clean[column] = pd.to_numeric(df_clean[column].astype(str).str.replace(' gal', '').str.strip(), errors='coerce')

    if 'maximum_seating' in df_clean.columns:
        df_clean['maximum_seating'] = pd.to_numeric(df_clean['maximum_seating'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')
    
    return df_clean

In [10]:
df_cleaned = clean_units(df_cleaned)

In [11]:
# Removing drivetrin info from trim_name column as it is already included in 'wheel_system', 'wheel_system_display', and 'drivetrain'.
def remove_drivetrain_from_trim(df):
    df_clean = df.copy()
    
    drivetrain_patterns = ['FWD', 'RWD', 'AWD', '4WD', '2WD', '4X4', '4x4']
    pattern_regex = r'\b(?:' + '|'.join(drivetrain_patterns) + r')\b'
    
    if 'trim_name' in df_clean.columns:
        df_clean['trim_name'] = df_clean['trim_name'].astype(str).str.replace(pattern_regex, '', case=False, regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
    
    return df_clean

In [12]:
df_cleaned = remove_drivetrain_from_trim(df_cleaned) 

In [13]:
# Removing text from 'power' and 'torque' columns.
# Currently both colums will show as "150 hp @ 5,600 RPM" or "150 lb-ft @ 4,000 RPM", this will extract text and leave only numeric values.
def clean_power_and_torque(df):
    df_clean = df.copy()

    if 'power' in df_clean.columns:
        
        df_clean['power'] = pd.to_numeric(df_clean['power'].astype(str).str.extract(r'(\d+)\s*hp', flags=re.IGNORECASE)[0], errors='coerce')

    if 'torque' in df_clean.columns:

        df_clean['torque'] = pd.to_numeric(df_clean['torque'].astype(str).str.extract(r'(\d+)\s*lb', flags=re.IGNORECASE)[0], errors='coerce')

    return df_clean

In [14]:
df_cleaned = clean_power_and_torque(df_cleaned)

In [15]:
def create_color_mapping(unique_colors):
    # Create mapping from detailed colors to base colors
    color_keywords = {
        'black': ['black', 'noir', 'ebony', 'onyx', 'carbon', 'charcoal', 'midnight', 'jet'],
        'white': ['white', 'pearl', 'snow', 'ivory', 'cream', 'arctic', 'alabaster', 'polar', 'frost'],
        'gray': ['gray', 'grey', 'graphite', 'slate', 'ash', 'gunmetal', 'titanium', 'steel', 'silver'],
        'silver': ['silver', 'sterling', 'aluminum', 'platinum', 'metallic'],
        'red': ['red', 'crimson', 'scarlet', 'cherry', 'ruby', 'burgundy', 'maroon', 'wine', 'rose'],
        'blue': ['blue', 'navy', 'sapphire', 'ocean', 'cobalt', 'azure', 'royal', 'sky', 'atlantic'],
        'green': ['green', 'emerald', 'forest', 'olive', 'sage', 'mint', 'jade', 'hunter'],
        'brown': ['brown', 'bronze', 'copper', 'espresso', 'mocha', 'chocolate', 'coffee', 'cocoa'],
        'beige': ['beige', 'tan', 'sand', 'khaki', 'champagne', 'taupe', 'desert'],
        'gold': ['gold', 'brass', 'honey', 'amber'],
        'yellow': ['yellow', 'lemon', 'mustard', 'sunshine'],
        'orange': ['orange', 'rust', 'sunset', 'tangerine', 'pumpkin', 'mango'],
        'purple': ['purple', 'violet', 'plum', 'lavender', 'mauve']
    }

    color_mapping = {}
    unmapped = []
    
    for color in unique_colors:
        if pd.isna(color) or color == 'nan':
            color_mapping[color] = np.nan
            continue

        color_lower = str(color).lower().strip()
        mapped = False

        if not mapped:
            for base_color, keywords in color_keywords.items():
                for keyword in keywords:
                    if keyword in color_lower.split():
                        color_mapping[color] = base_color
                        mapped = True
                        break
                if mapped:
                    break
        if not mapped:
            color_mapping[color] = 'other'
            unmapped.append(color)
    
    return color_mapping, unmapped

In [16]:
# Create mapping for exterior colors
unique_colors = df_cleaned['exterior_color'].unique()
color_map, unmapped_colors = create_color_mapping(unique_colors)
df_cleaned['exterior_color_base'] = df_cleaned['exterior_color'].map(color_map)

# Create mapping for interior colors
unique_interior_colors = df_cleaned['interior_color'].unique()
interior_color_map, interior_unmapped = create_color_mapping(unique_interior_colors)
df_cleaned['interior_color_base'] = df_cleaned['interior_color'].map(interior_color_map)

df_cleaned['interior_color_metallic'] = df_cleaned['interior_color'].astype(str).str.contains('metallic', case=False, na=False).astype(int)
df_cleaned['interior_color_pearl'] = df_cleaned['interior_color'].astype(str).str.contains('pearl', case=False, na=False).astype(int)
df_cleaned['interior_color_matte'] = df_cleaned['interior_color'].astype(str).str.contains('matte|flat', case=False, na=False).astype(int)

In [17]:
color_mappings = {
    'exterior_color_map': {k: v for k, v in color_map.items() if pd.notna(k)},
    'interior_color_map': {k: v for k, v in interior_color_map.items() if pd.notna(k)}
}

with open('color_mappings.json', 'w') as f:
    json.dump(color_mappings, f)

In [ ]:
def process_full_dataset(input_file = 'data/raw/used_cars_data.csv', 
                        output_file = 'data/processed/used_cars_data_cleaned.csv', 
                        chunksize = 50000):
    with open('color_mappings.json', 'r') as f:
        color_mappings = json.load(f)
    
    exterior_color_map = color_mappings['exterior_color_map']
    interior_color_map = color_mappings['interior_color_map']

    for i, chunk in enumerate(pd.read_csv(input_file, chunksize = chunksize)):
        chunk = lowercase_text_columns(chunk)
        chunk = clean_units(chunk)
        chunk = remove_drivetrain_from_trim(chunk)
        chunk = clean_power_and_torque(chunk)

        chunk['exterior_color_base'] = chunk['exterior_color'].map(exterior_color_map).fillna('other')
        chunk['interior_color_base'] = chunk['interior_color'].map(interior_color_map).fillna('other')

        for col in ['exterior_color', 'interior_color']:
            chunk[f'{col}_metallic'] = chunk[col].astype(str).str.contains('metallic', case = False, na = False).astype(int)
            chunk[f'{col}_pearl'] = chunk[col].astype(str).str.contains('pearl', case = False, na = False).astype(int)

        chunk.to_csv(output_file, mode = 'w' if i == 0 else 'a', header = (i == 0), index = False)

        print(f'Processed chunk {i + 1}')

In [ ]:
# Processing the full dataset in chunks to manage memory usage
process_full_dataset()

In [ ]:
# Visualization for Part D showing unique amounts of exterior and interior colors.
unique_exterior_count = df_sample['exterior_color'].nunique()
unique_interior_count = df_sample['interior_color'].nunique()

color_types = ['Exterior Colors', 'Interior Colors']
unique = [unique_exterior_count, unique_interior_count]

plt.figure(figsize = (8, 6))
bars = plt.bar(color_types, unique, color = ['#4B5563', '#6B7280'])

plt.title('Count of Unique Colors', fontsize=16)
plt.ylabel('Number of Unique Values', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, int(yval), va='bottom', ha='center', fontsize=12)

plt.savefig('unique_colors.png')
plt.show()

In [ ]:
# Feature Importance of model by percentage, used to prove hypothesis in part D Hypothesis Verification

import pickle 
import pandas as pd

with open('../backend/models/saved/vehicle_predictor_model_3m.pkl', 'rb') as f:
    model = pickle.load(f)

model_dict = model['model']
features = model['feature_cols']

feature_importance = model_dict.feature_importances_

# Create DataFrame of features
feature_importance_df = pd.DataFrame({
    'feature': features,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

feature_importance_df['importance percentage'] = (
    feature_importance_df['importance'] / feature_importance_df['importance'].sum() * 100
)

print(feature_importance_df[['feature', 'importance percentage']].head(20))